In [54]:
import unityneuro.render as urn
import pandas as pd
import numpy as np
import seaborn as sns
import time as time

In [57]:
def labs():
    lab_number_map = {'cortexlab': 'Lab 1', 'mainenlab': 'Lab 2', 'churchlandlab': 'Lab 3',
                      'angelakilab': 'Lab 4', 'wittenlab': 'Lab 5', 'hoferlab': 'Lab 6',
                      'mrsicflogellab': 'Lab 6', 'danlab': 'Lab 7', 'zadorlab': 'Lab 8',
                      'steinmetzlab': 'Lab 9', 'churchlandlab_ucla': 'Lab 10', 'hausserlab': 'Lab 11'}
    institution_map = {'cortexlab': 'UCL', 'mainenlab': 'CCU', 'zadorlab': 'CSHL (Z)',
                       'churchlandlab': 'CSHL (C)', 'angelakilab': 'NYU',
                       'wittenlab': 'Princeton', 'hoferlab': 'SWC', 'mrsicflogellab': 'SWC',
                       'danlab': 'Berkeley', 'steinmetzlab': 'UW', 'churchlandlab_ucla': 'UCLA', 'hausserlab': 'UCL'}
    colors = np.concatenate([sns.color_palette("Dark2"), sns.color_palette('Set2')[0:2]])
    institutions = ['UCL', 'CCU', 'CSHL (C)', 'NYU', 'Princeton', 'SWC', 'Berkeley', 'CSHL (Z)',
                    'UW', 'UCLA']
    institution_colors = {}
    for i, inst in enumerate(institutions):
        institution_colors[inst] = colors[i]
    return lab_number_map, institution_map, institution_colors

In [86]:
inst2text = {'cortexlab': 'Cortex Lab (UCL)', 'mainenlab': 'Mainen Lab (CCU)', 'zadorlab': 'Zador Lab (CSHL)',
                       'churchlandlab': 'Churchland Lab (CSHL)', 'angelakilab': 'Angelaki Lab (NYU)',
                       'wittenlab': 'Witten Lab (Princeton)', 'hoferlab': 'Hofer Lab (SWC)', 'mrsicflogellab': 'Mrsic-Flogel Lab (SWC)',
                       'danlab': 'Dan Lab (Berkeley)', 'steinmetzlab': 'Steinmetz Lab (UW)', 'churchlandlab_ucla': 'Churchland Lab (UCLA)', 'hausserlab': 'Hausser Lab (UCL)'}

In [58]:
lab_number_map, institution_map, institution_colors = labs()

In [44]:
def lab2color(lab):
    color_array = institution_colors[institution_map[lab]]
    return '#%02x%02x%02x' % (int(color_array[0]*255), int(color_array[1]*255), int(color_array[2]*255))

In [2]:
data = pd.read_csv('./lab_clusters.csv', index_col=0)
data = data[data['label']==1]
print(data.shape)

(32766, 7)


In [3]:
data.head()

,eid,pid,ml_ccf,ap_ccf,dv_ccf,lab,label
1,6713a4a7-faed-4df2-acab-ee4e63326f8d,56f2a378-78d2-4132-b3c8-8c1ba82be598,3450.0,7937.0,3927.0,angelakilab,1.0
3,6713a4a7-faed-4df2-acab-ee4e63326f8d,56f2a378-78d2-4132-b3c8-8c1ba82be598,3450.0,7937.0,3927.0,angelakilab,1.0
10,6713a4a7-faed-4df2-acab-ee4e63326f8d,56f2a378-78d2-4132-b3c8-8c1ba82be598,3450.0,7933.0,3908.0,angelakilab,1.0
13,6713a4a7-faed-4df2-acab-ee4e63326f8d,56f2a378-78d2-4132-b3c8-8c1ba82be598,3450.0,7928.0,3888.0,angelakilab,1.0
16,6713a4a7-faed-4df2-acab-ee4e63326f8d,56f2a378-78d2-4132-b3c8-8c1ba82be598,3450.0,7928.0,3888.0,angelakilab,1.0


In [4]:
urn.setup()

(URN) connected to server
Login sent with ID: Dan


In [82]:
urn.clear()

areas = [22, 31, 44, 95, 184, 247, 254, 343, 453, 500, 512, 541, 623, 669, 677, 698, 703, 714, 895, 922, 972, 1057, 1089]

area_vis = {}
area_mat = {}
area_alpha = {}

for area in areas:
    area_vis[area] = True
    area_mat[area] = 'transparent-unlit'
    area_alpha[area] = 0.15


# Load brain areas     
urn.set_area_visibility(area_vis)
urn.set_area_material(area_mat)
urn.set_area_alpha(area_alpha)

In [60]:
labs = np.unique(data['lab'])

In [92]:
urn.set_camera_rotation(45,0,0)
time.sleep(5)

for lab in labs:
    ldata = data[data['lab']==lab]
    lab_color = lab2color(lab).upper()

    anames = []
    apos = []
    acol = []
    ashape = []

    # we're going to prefill a bunch of dictionaries for each probe, then we'll pop them into view one at a time (less dramatic then making all neurons appear at once)
    upid = np.unique(ldata['pid'].values)

    for pid in upid:
        pdata = ldata[ldata['pid']==pid]

        neuron_names = []
        neuron_pos = {}
        neuron_col = {}
        neuron_shape = {}

        for i, row in pdata.iterrows():
            name = 'n'+str(i)
            neuron_names.append(name)
            neuron_pos[name] = ([row['ml_ccf'], row['ap_ccf'], row['dv_ccf']])
            neuron_shape[name] = 'sphere'
            neuron_col[name] = lab_color

        anames.append(neuron_names)
        apos.append(neuron_pos)
        acol.append(neuron_col)
        ashape.append(neuron_shape)

    urn.clear_texts()
    # Setting the text names, positions, sizes, and colors
    urn.create_text(['t1'])
    urn.set_text({'t1':inst2text[lab]})
    urn.set_text_positions({'t1':[-0.6,0.5]})
    urn.set_text_colors({'t1':lab_color})

    # now iterate through probes
    urn.clear_neurons()
    for i in np.arange(0,len(anames)):
        urn.create_neurons(anames[i])
        urn.set_neuron_positions(apos[i])
        urn.set_neuron_shapes(ashape[i])
        urn.set_neuron_colors(acol[i])
        time.sleep(0.02)

    time.sleep(2.5)

(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan
(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan


packet queue is empty, aborting


(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan


packet queue is empty, aborting


(URN) disconnected from server
(URN) connected to server
Login sent with ID: Dan


In [11]:
print(labs)

['angelakilab' 'churchlandlab' 'churchlandlab_ucla' 'cortexlab' 'danlab'
 'hausserlab' 'hoferlab' 'mainenlab' 'mrsicflogellab' 'steinmetzlab'
 'wittenlab' 'zadorlab']
